## Converting observed station Data into PCraster Timeseries format.
Coded by: Dinesh Joshi \
Email: joshidinesh0227@gmail.com

In [1]:
## run this cell once to install openpyxl
!pip install openpyxl

   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.0 kB 660.6 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.0 kB 393.8 kB/s eta 0:00:01
   ----------------- -------------------- 112.6/250.0 kB 819.2 kB/s eta 0:00:01
   ------------------------ ------------- 163.8/250.0 kB 893.0 kB/s eta 0:00:01
   ---------------------------------------  245.8/250.0 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 250.0/250.0 kB 1.0 MB/s eta 0:00:00


## Provide Inputs in below.

In [2]:
BaseTemperatureStationElevation= 720 # Provide elevation of Base temperature station
HigestElevationTemperatureStation= 2000 # Provide elevation of temperature stations at higest availabe altitude

###  In the next code cell, we will load the observed station data and convert it into pcraster timeseries format in the desired resolution as mask file. Run the cell below to execute this code. You can check Prec, Tmax, Tmin subfolders inside the main folder for the resultant data. The processing will take time depending on the amount of data and size of study area.

In [3]:
# importing the required libraries
import pandas as pd
import numpy as np
from pcraster import *
from pcraster.framework import *

# importing Data
df=pd.read_excel("Station Data.xlsx",index_col=0)

df['Date'] = pd.to_datetime(df['Date'])
# Add a new column 'Month' to the DataFrame
df['Month'] = df['Date'].dt.month

# Group by month and calculate the average lapse rate for each month
Tmax_monthly_lapse_rates = df.groupby('Month').apply(lambda x: ((x['Tmax1'] - x['Tmax2'])/(HigestElevationTemperatureStation-BaseTemperatureStationElevation)).mean())
Tmin_monthly_lapse_rates = df.groupby('Month').apply(lambda x: ((x['Tmin1'] - x['Tmin2'])/(HigestElevationTemperatureStation-BaseTemperatureStationElevation)).mean())

precColumns=[]
for i in df.columns:
    if i[:4]== "prec":
        precColumns.append(i)

NColumns= len(precColumns)+1                
with open('ObservedPrecipitationTimeseriesData.tss', 'w') as f:
    f.write("Observed Precipitation time series scaler data")
    f.write("\n")
    f.write(str(NColumns))
    f.write("\n")
    f.write("timestep")
    f.write("\n")
    for i in range(1,NColumns):
        f.write(str(i))
        f.write("\n")
    for timestep in range(len(df)):
        f.write(str(timestep+1)+' ')
        for precColumn in precColumns:
            f.write(str(df[precColumn][timestep])+ ' ')
        f.write(str("\n"))
# change the current working directory to GDM Training'
import os
os.chdir("..")
os.chdir("..")

# Creating a pcraster map file for station id coordinates¶
# set the paths to the input and output files
coordinates_file = './Data Preprocessing/Observed Station Data/stationid.txt'
mask_file = './Inputs/mask.map'
output_file = './Data Preprocessing/Observed Station Data/stationid.map'

# build the command string with the file paths
command = ['col2map', '-N', coordinates_file, output_file, '--clone', mask_file]

# execute the command using subprocess.run
subprocess.run(command)

# interpolating using inverse distance weighting
class InterpolateRainfall(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Map with coordinates
        self.coordinates = self.readmap("./Data Preprocessing/Observed Station Data/stationid") 
        # Boolean mask for IDW 
        self.mask = self.readmap("./Inputs/mask")            
        # Dem for Temperature lapse rate
        self.DEM= self.readmap("./Inputs/dem")
        #variable for tss file
        self.precipTSS = "./Data Preprocessing/Observed Station Data/ObservedPrecipitationTimeseriesData.tss"      
        #initializing incremental timestep 
        self.IncrementalTimestep= 1
       
    def dynamic(self):
        Timestep=int(self.IncrementalTimestep)
        # Reading precipitation at coordinates and make dynamic map stations
        precipAtStation = timeinputscalar(self.precipTSS,self.coordinates)
        #IDW interpolation
        precipIDW = inversedistance(self.mask,precipAtStation,2,0,0)
        
        # Interpolating temperature using temperature lapse rate.
        BTmax=df['Tmax1'][Timestep-1]-((self.DEM-BaseTemperatureStationElevation)*Tmax_monthly_lapse_rates[df["Month"][Timestep-1]])
        BTmin=df['Tmin1'][Timestep-1]-((self.DEM-BaseTemperatureStationElevation)*Tmin_monthly_lapse_rates[df["Month"][Timestep-1]])
        HTmax=df['Tmax1'][Timestep-1]-((self.DEM-BaseTemperatureStationElevation)*0.006)
        HTmin=df['Tmin1'][Timestep-1]-((self.DEM-BaseTemperatureStationElevation)*0.006)   
        
        Tmax=ifthenelse(self.DEM < HigestElevationTemperatureStation, BTmax, HTmax)
        Tmin=ifthenelse(self.DEM < HigestElevationTemperatureStation, BTmin, HTmin)
        
        self.IncrementalTimestep= self.IncrementalTimestep+1
        
        self.report(precipIDW,"./Prec/prec") ## provide path for the precipitation output
        self.report(Tmax,"./Tmax/tmax")
        self.report(Tmin,"./Tmin/tmin")
myModel = InterpolateRainfall("./Inputs/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=len(df), firstTimestep=1) ##Adjust first and last time step
dynModelFw.run()
os.chdir("./Data Preprocessing/Observed Station Data")

C:\Users\Hp\AppData\Local\Temp\ipykernel_8092\2891329538.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Tmax_monthly_lapse_rates = df.groupby('Month').apply(lambda x: ((x['Tmax1'] - x['Tmax2'])/(HigestElevationTemperatureStation-BaseTemperatureStationElevation)).mean())
C:\Users\Hp\AppData\Local\Temp\ipykernel_8092\2891329538.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Tmin_monthly_lapse_rate

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 